# Imports

In [15]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 255)
from bs4 import BeautifulSoup
import nltk
from nltk.stem.porter import *
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
from pathlib import Path
nltk.download('punkt')
from dotenv import load_dotenv
import os

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\t1nipun\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [16]:
%cd "c:\\Users\\t1nipun\\Documents\\esa-data-bank_banque-donnees-ees"
ROOT_PATH = os.getcwd()
from berdi.Database_Connection_Files.connect_to_sqlserver_database import connect_to_db

c:\Users\t1nipun\Documents\esa-data-bank_banque-donnees-ees


In [17]:
nltk.download('stopwords', quiet=True)

True

`esa_csvs_raw.txt` is a tab-separated file of the raw data from the MySQL database. You will need to create this file if you want to include new data. The file contains the name of the CSV file as well as the contents of the CSV file (of the table). You need to make it tab-separated because MSSQL will not separate things properly if you use a comma because the CSV contents include commas. If you *really* want to use a comma, there is a way to do it where there will be quotes added to separate the fields, but you will need to look up how to do it.

In [18]:
load_dotenv(
    dotenv_path= str(ROOT_PATH) + "\\berdi\\Database_Connection_Files\\.env", override=True
)
conn = connect_to_db()

df = pd.read_sql('''SELECT csvFileName, csvText FROM [DS_TEST].[BERDI].csvs;''', conn)
df.to_csv(str(ROOT_PATH) + "\\data\\processed\\csvs\\esa_csvs_raw.txt", index = False, sep = '\t', encoding = 'utf-8-sig')

In [19]:
df_raw = pd.read_csv(str(ROOT_PATH) + "\\data\\processed\\csvs\\esa_csvs_raw.txt", sep='\t')

In [20]:
df_raw.iloc[0]

csvFileName                                                                                                                                                                                                                                        4166218_10_lattice-v_1.csv
csvText        [["Residual Cumulative Effect","Residual Cumulative Effects Characterization","","","","","",""],["Residual Cumulative Effect","Magnitude","Extent Geographic","Timing","Duration","Frequency","Reversibility","Likelihood"],["Change in Habitat","","","",...
Name: 0, dtype: object

In [21]:
df_raw['csvText'] = df_raw['csvText'].str.slice(0, 30_000) # limit to 30k chars since Excel can't handle more than 32k

In [22]:
assert (df_raw['csvText'].str.len()).max() <= 30_000 # sanity check

In [23]:
df_index = pd.read_csv(str(ROOT_PATH) + "\\data\\interim\\Intermediate_Index_Files\\Table_Fig_Alignment_Index.csv", encoding='utf-8-sig')

In [ ]:
# df_index.rename(columns={\"Old CSV Filename\": \"csvFileName\"}, inplace=True)

In [24]:
df_index.head(2)

,Application Filing Date,Application ID,Application Name,Application Short Name,Application Type,Application URL,Commodity,Company Name,Consultant Name,Content Type,Data ID,Decision URL,Document Number,ESA Folder URL,ESA Section(s),File Name,Hearing Order,PDF Download URL,PDF Page Count,PDF Page Number,Pipeline Location,Pipeline Status,Regulatory Instrument(s),Title,Topics,csvFileName,figureNumber,tableNumber
0,11/18/2021,59480,Application for the NEBC Connector Project,NEBC Connector,Large Projects (over 40 km),https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4158245,Gas,NorthRiver Midstream NEBC Connector GP Inc.,Stantec,Table,4166218,NaN,A7Y7F7,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4166216,"APPENDIX 1-1 ENVIRONMENTAL AND SOCIO-ECONOMIC ASSESSMENT, 7.0 ASSESSMENT OF POTENTIAL EFFECTS ON WILDLIFE AND WILDLIFE HABITAT",C16186-6,OH-001-2022,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/4166218,20,10.0,"British Columbia, Alberta",Applied,NaN,Table 7.28 Residual Cumulative Effects on Wildlife and Wildlife Habitat,"['Wildlife', 'Human']",4166218_10_lattice-v_1.csv,NaN,1.0
1,11/18/2021,59480,Application for the NEBC Connector Project,NEBC Connector,Large Projects (over 40 km),https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4158245,Gas,NorthRiver Midstream NEBC Connector GP Inc.,Stantec,Table,4166218,NaN,A7Y7F7,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4166216,"APPENDIX 1-1 ENVIRONMENTAL AND SOCIO-ECONOMIC ASSESSMENT, 7.0 ASSESSMENT OF POTENTIAL EFFECTS ON WILDLIFE AND WILDLIFE HABITAT",C16186-6,OH-001-2022,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/4166218,20,4.0,"British Columbia, Alberta",Applied,NaN,Table 7.27 Change in Linear Features Density within the Wildlife Regional Assessment Area for Project Case and Future Case,"['Wildlife', 'Human']",4166218_4_lattice-v_1.csv,NaN,1.0


In [25]:
df_index.iloc[133:135] # sanity check

,Application Filing Date,Application ID,Application Name,Application Short Name,Application Type,Application URL,Commodity,Company Name,Consultant Name,Content Type,Data ID,Decision URL,Document Number,ESA Folder URL,ESA Section(s),File Name,Hearing Order,PDF Download URL,PDF Page Count,PDF Page Number,Pipeline Location,Pipeline Status,Regulatory Instrument(s),Title,Topics,csvFileName,figureNumber,tableNumber
133,11/18/2021,59480,Application for the NEBC Connector Project,NEBC Connector,Large Projects (over 40 km),https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4158245,Gas,NorthRiver Midstream NEBC Connector GP Inc.,"Stantec, AiM Land Services Ltd.",Table,4166301,NaN,A7Y7F8,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4166216,"APPENDIX 1-1 ENVIRONMENTAL AND SOCIO-ECONOMIC ASSESSMENT, 8.0 ASSESSMENT OF POTENTIAL EFFECTS ON FISH AND FISH HABITAT, 9.0 ASSESSMENT OF POTENTIAL EFFECTS ON GREENHOUSE GASES, 10.0 ASSESSMENT OF POTENTIAL EFFECTS ON AIR QUALITY, 11.0 ASSESSMENT OF PO...",C16186-7,OH-001-2022,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/4166301,531,334.0,"British Columbia, Alberta",Applied,NaN,Table 17.9 Educational Attainment – 2016,"['Land', 'Air', 'Water', 'Wildlife', 'Human', 'Traditional Knowledge']",4166301_334_lattice-v_1.csv,NaN,1.0
134,11/18/2021,59480,Application for the NEBC Connector Project,NEBC Connector,Large Projects (over 40 km),https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4158245,Gas,NorthRiver Midstream NEBC Connector GP Inc.,"Stantec, AiM Land Services Ltd.",Table,4166301,NaN,A7Y7F8,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4166216,"APPENDIX 1-1 ENVIRONMENTAL AND SOCIO-ECONOMIC ASSESSMENT, 8.0 ASSESSMENT OF POTENTIAL EFFECTS ON FISH AND FISH HABITAT, 9.0 ASSESSMENT OF POTENTIAL EFFECTS ON GREENHOUSE GASES, 10.0 ASSESSMENT OF POTENTIAL EFFECTS ON AIR QUALITY, 11.0 ASSESSMENT OF PO...",C16186-7,OH-001-2022,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/4166301,531,335.0,"British Columbia, Alberta",Applied,NaN,Table 17.9 Educational Attainment – 2016,"['Land', 'Air', 'Water', 'Wildlife', 'Human', 'Traditional Knowledge']",4166301_335_lattice-v_1.csv,NaN,1.0


In [26]:
df_merged = df_raw[df_raw['csvFileName'].isin(list(df_index['csvFileName']))].reset_index(drop=True)

In [27]:
df_joined = pd.merge(df_index, df_merged, how='left', on='csvFileName')

In [28]:
df_joined.head(2)

,Application Filing Date,Application ID,Application Name,Application Short Name,Application Type,Application URL,Commodity,Company Name,Consultant Name,Content Type,Data ID,Decision URL,Document Number,ESA Folder URL,ESA Section(s),File Name,Hearing Order,PDF Download URL,PDF Page Count,PDF Page Number,Pipeline Location,Pipeline Status,Regulatory Instrument(s),Title,Topics,csvFileName,figureNumber,tableNumber,csvText
0,11/18/2021,59480,Application for the NEBC Connector Project,NEBC Connector,Large Projects (over 40 km),https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4158245,Gas,NorthRiver Midstream NEBC Connector GP Inc.,Stantec,Table,4166218,NaN,A7Y7F7,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4166216,"APPENDIX 1-1 ENVIRONMENTAL AND SOCIO-ECONOMIC ASSESSMENT, 7.0 ASSESSMENT OF POTENTIAL EFFECTS ON WILDLIFE AND WILDLIFE HABITAT",C16186-6,OH-001-2022,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/4166218,20,10.0,"British Columbia, Alberta",Applied,NaN,Table 7.28 Residual Cumulative Effects on Wildlife and Wildlife Habitat,"['Wildlife', 'Human']",4166218_10_lattice-v_1.csv,NaN,1.0,"[[""Residual Cumulative Effect"",""Residual Cumulative Effects Characterization"","""","""","""","""","""",""""],[""Residual Cumulative Effect"",""Magnitude"",""Extent Geographic"",""Timing"",""Duration"",""Frequency"",""Reversibility"",""Likelihood""],[""Change in Habitat"","""","""","""",..."
1,11/18/2021,59480,Application for the NEBC Connector Project,NEBC Connector,Large Projects (over 40 km),https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4158245,Gas,NorthRiver Midstream NEBC Connector GP Inc.,Stantec,Table,4166218,NaN,A7Y7F7,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/4166216,"APPENDIX 1-1 ENVIRONMENTAL AND SOCIO-ECONOMIC ASSESSMENT, 7.0 ASSESSMENT OF POTENTIAL EFFECTS ON WILDLIFE AND WILDLIFE HABITAT",C16186-6,OH-001-2022,https://apps.cer-rec.gc.ca/REGDOCS/File/Download/4166218,20,4.0,"British Columbia, Alberta",Applied,NaN,Table 7.27 Change in Linear Features Density within the Wildlife Regional Assessment Area for Project Case and Future Case,"['Wildlife', 'Human']",4166218_4_lattice-v_1.csv,NaN,1.0,"[[""Linear Features Density Class (km\/km<s>2<\/s>)"",""Baseline Case"","""",""Project Case"","""","""","""",""Future Case"","""","""",""""],[""Linear Features Density Class (km\/km<s>2<\/s>)"",""Amount of Habitat Available"","""",""Amount of Habitat Available"","""",""Change from Ba..."


In [29]:
len(df_joined) # 502

1032

In [30]:
df_joined.columns

Index(['Application Filing Date', 'Application ID', 'Application Name',
       'Application Short Name', 'Application Type', 'Application URL',
       'Commodity', 'Company Name', 'Consultant Name', 'Content Type',
       'Data ID', 'Decision URL', 'Document Number', 'ESA Folder URL',
       'ESA Section(s)', 'File Name', 'Hearing Order', 'PDF Download URL',
       'PDF Page Count', 'PDF Page Number', 'Pipeline Location',
       'Pipeline Status', 'Regulatory Instrument(s)', 'Title', 'Topics',
       'csvFileName', 'figureNumber', 'tableNumber', 'csvText'],
      dtype='object')

In [ ]:
del df_raw
del df_index
del df_merged

## Adding New Projects to the Index File

In [31]:
df_joined.rename(columns={"csvText": "text"}, inplace=True)
df_joined['label'] = -1

In [32]:
df_joined["text"] = df_joined["Title"] + " " + df_joined["text"]

In [33]:
len(df_joined)

1032

In [34]:
df_joined['text'] = df_joined['text'].astype(str).apply(lambda x: BeautifulSoup(x, "html.parser").get_text(separator=' ')) # remove html tags
df_joined['text'] = df_joined['text'].replace('[^a-zA-Z0-9 ]', ' ', regex=True) # remove all non-alpha-numeric characters
df_joined['text'] = df_joined['text'].replace('\w{25,}', ' ', regex=True)
df_joined['text'] = df_joined['text'].replace('cid\d+', ' ', regex=True)
df_joined['text'] = df_joined['text'].replace(' s ', ' ', regex=True)
df_joined['text'] = df_joined['text'].replace(' +', ' ', regex=True) # remove all extra spaces in text

In [35]:
df_joined.text[0]

'Table 7 28 Residual Cumulative Effects on Wildlife and Wildlife Habitat Residual Cumulative Effect Residual Cumulative Effects Characterization Residual Cumulative Effect Magnitude Extent Geographic Timing Duration Frequency Reversibility Likelihood Change in Habitat Overall cumulative effect L M RAA LR CA CR LT C R I L Contribution from the Project to the overall cumulative effect The Project will result in added residual effects with the loss of effective wildlife habitat ranging from 21 to 342 ha depending on the focal species or species group Table 7 19 during the construction phase This loss of effective habitat is characterized as low to moderate in magnitude depending on the focal species or species group 1 3 to 5 2 change in habitat availability within the LAA The offset plan will include planting of previously cleared areas The Project u2019s contribution to cumulative effects on wildlife habitat will be offset once this vegetation becomes established Change in Mortality Risk

In [ ]:
df_joined.to_csv(str(ROOT_PATH) + '\\data\\interim\\Intermediate_Index_Files\\esa_index_with_table_text_no_labels.csv', index=False, encoding='utf-8-sig')

In [ ]:
from sklearn.model_selection import train_test_split
train_df, df_for_labeling = train_test_split(df_joined, test_size=1000, random_state=42, shuffle=True)

In [ ]:
print(len(train_df))
print(len(df_for_labeling))

In [ ]:
train_df.head(2)

In [ ]:
df_for_labeling.head(2)

In [ ]:
df_for_labeling.to_csv(str(ROOT_PATH / 'data/interim/Intermediate_Index_Files/esa_index_for_labeling.csv'), index=False, encoding='utf-8')